# 4.2.0 손실함수

### 4.2.1 오차제곱합
---

In [ ]:
import numpy as np

In [4]:
#손실함수-오차제곱합
def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [5]:
t = [0,0,1,0,0,0,0,0,0,0]
y = [.1,.05,.6,.0,.05,.1,.0,.1,.0,.0]

sum_squares_error(np.array(y), np.array(t))

0.09750000000000003

In [7]:
y = [.1,.05,.1,.0,.05,.1,.0,.6,.0,.0]
sum_squares_error(np.array(y), np.array(t))

0.5975

### 4.2.2 교차 엔트로피 오차
---

In [8]:
#손실함수-교차 엔트로피 오차
#delta를 y(신경망의 출력) 값에 더해주는 이유 : log0은 정의 할 수 없을 뿐더러 이론적으로 -무한대에 수렴하기 때문에, 이것을 방지하여 반드시
#0일 수 없도록 아주 작은 상수(1e-7 = 10^-7)을 더해준 것.
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y+delta))

In [9]:
t = [0,0,1,0,0,0,0,0,0,0]
y = [.1,.05,.6,.0,.05,.1,.0,.1,.0,.0]

cross_entropy_error(np.array(y),np.array(t))

0.510825457099338

In [10]:
y = [.1,.05,.1,.0,.05,.1,.0,.6,.0,.0]
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546